In [4]:
from Covid19Environment import GraphicCovid19Environment, Covid19Environment, States, Actions
import numpy as np
import random
import math

In [5]:
# Parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.01            # Minimum exploration probability 
decay_rate = 0.01             # Exponential decay rate for exploration prob

learning_rate = 0.7
discount_rate = 0.618

steps = 1
total_episodes = 3

world_size = (5, 5)

# Init Q table
q_table = np.zeros((5, 5, 3))
# If you see border, then you should not go forward
for i in range(5):
    q_table[i][States.BORDER.value][Actions.FORWARD.value] = -math.inf
#print(q_table)

for episode in range(total_episodes):
    done = False
    total_reward = 0
    env = Covid19Environment(q_table, epsilon, world_size)
    for s in range(steps):
        done, new_reward = env.step()
        total_reward += new_reward
        # If done : finish episode
        if done == True:
            break
    print(episode, " Finished: ", s, ", reward: ", total_reward, "Random/total actions: ", (env.random_actions*100)/env.total_actions)
        # Reduce epsilon (because we need less and less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)

np.set_printoptions(precision=2, suppress=True)
print("\n\nQ Table:\n", q_table)

0  Finished:  299 , reward:  -300 Random/total actions:  100.0
1  Finished:  299 , reward:  -300 Random/total actions:  100.0
2  Finished:  299 , reward:  -300 Random/total actions:  99.33333333333333
3  Finished:  299 , reward:  -300 Random/total actions:  98.0
4  Finished:  299 , reward:  -300 Random/total actions:  98.0
5  Finished:  299 , reward:  -300 Random/total actions:  96.33333333333333
6  Finished:  299 , reward:  -300 Random/total actions:  94.33333333333333
7  Finished:  299 , reward:  -300 Random/total actions:  97.0
8  Finished:  299 , reward:  -300 Random/total actions:  93.33333333333333
9  Finished:  299 , reward:  -300 Random/total actions:  88.66666666666667
10  Finished:  299 , reward:  -300 Random/total actions:  87.33333333333333
11  Finished:  299 , reward:  -300 Random/total actions:  90.66666666666667
12  Finished:  299 , reward:  -300 Random/total actions:  88.33333333333333
13  Finished:  299 , reward:  -300 Random/total actions:  88.0
14  Finished:  299 , r

114  Finished:  299 , reward:  -300 Random/total actions:  32.333333333333336
115  Finished:  299 , reward:  -300 Random/total actions:  35.0
116  Finished:  299 , reward:  -300 Random/total actions:  30.0
117  Finished:  299 , reward:  -300 Random/total actions:  34.0
118  Finished:  299 , reward:  -300 Random/total actions:  29.333333333333332
119  Finished:  299 , reward:  -300 Random/total actions:  39.0
120  Finished:  299 , reward:  -300 Random/total actions:  27.333333333333332
121  Finished:  299 , reward:  -300 Random/total actions:  30.0
122  Finished:  299 , reward:  -300 Random/total actions:  28.666666666666668
123  Finished:  299 , reward:  -300 Random/total actions:  32.666666666666664
124  Finished:  299 , reward:  -300 Random/total actions:  29.0
125  Finished:  299 , reward:  -300 Random/total actions:  27.333333333333332
126  Finished:  299 , reward:  -300 Random/total actions:  33.0
127  Finished:  299 , reward:  -300 Random/total actions:  32.666666666666664
128  F

235  Finished:  299 , reward:  -300 Random/total actions:  12.666666666666666
236  Finished:  299 , reward:  -300 Random/total actions:  12.0
237  Finished:  299 , reward:  -300 Random/total actions:  8.666666666666666
238  Finished:  299 , reward:  -300 Random/total actions:  7.666666666666667
239  Finished:  299 , reward:  -300 Random/total actions:  12.333333333333334
240  Finished:  299 , reward:  -300 Random/total actions:  11.333333333333334
241  Finished:  299 , reward:  -300 Random/total actions:  10.0
242  Finished:  299 , reward:  -300 Random/total actions:  12.333333333333334
243  Finished:  299 , reward:  -300 Random/total actions:  10.333333333333334
244  Finished:  299 , reward:  -300 Random/total actions:  9.0
245  Finished:  299 , reward:  -300 Random/total actions:  10.333333333333334
246  Finished:  299 , reward:  -300 Random/total actions:  8.666666666666666
247  Finished:  299 , reward:  -300 Random/total actions:  8.0
248  Finished:  299 , reward:  -300 Random/tota

In [3]:
# Graphical representation of the game
env = GraphicCovid19Environment(True, q_table)

PlottingGrid
FinishedPlotting
add_house
add_house
add_kiwi
add_kiwi
add_survivor
add_survivor
[-2.62 -2.62 -2.62]
States.NOTHING States.NOTHING
[-2.62 -2.62 -2.62]
States.NOTHING States.NOTHING
[-2.62 -2.62 -2.62]
States.NOTHING States.NOTHING
[-2.62 -2.62 -2.62]
States.NOTHING States.NOTHING
[-2.62 -2.62 -2.62]
States.NOTHING States.NOTHING
[-2.62 -2.62 -2.62]
States.NOTHING States.NOTHING
[-2.62 -2.62 -2.62]
States.NOTHING States.NOTHING
[-2.62 -2.62 -2.62]
States.NOTHING States.NOTHING
[-2.62 -2.62 -2.62]
States.NOTHING States.NOTHING
[-2.62 -2.62 -2.62]
States.NOTHING States.NOTHING
Ooops, something is wrong
Traceback (most recent call last):
  File "D:\NTNU\TDT4265 Machine Vision\PhysicalDistancingRL\physicaldistRL\Covid19Environment.py", line 149, in __init__
    self.listen_for_keys()
  File "D:\NTNU\TDT4265 Machine Vision\PhysicalDistancingRL\physicaldistRL\Covid19Environment.py", line 159, in listen_for_keys
    k = self.win.checkKey()
  File "C:\Users\Maria\Anaconda3\lib\site